# Multi-instrument, multi-wavelength study of high energy sources with the Virtual Observatory

François Bonnarel¹, Caroline Bot¹, Françoise Genova¹, René Goosmann¹, Ada Nebot¹.

1. Université de Strasbourg, CNRS, Observatoire Astronomique de Strasbourg, UMR 7550, F-67000, Strasbourg, France

This Notebook is based on the Euro-VO "Multi-instrument, multi-wavelength study of high energy sources with the Virtual Observatory" tutorial http://www.euro-vo.org/sites/default/files/documents/tutorial-highenergy_2016Nov.pdf. 

Developed by Caroline Bot, François Bonnarel, René Goosmann and Françoise Genova. 
Adapted to a Jupyter Notebook by Ada Nebot. 

<img src="images/cds.png" alt="Drawing" style="width: 200px;"/>

***

## Introduction

This tutorial demonstrates how to use several standard tools of the Virtual Observatory (VO) for data mining and multiple waveband data analysis. The step-by-step application below focuses on applications in the gamma-ray and high energy domain, but also involves observational data from other wavebands. The user may explore how to:

1. query astronomical catalogues in the gamma-ray and high energy spectral band using VO tools
2. cross-correlate catalogues to find an object at different photon energy bands
3. apply selection criteria when extracting sources from a catalogue
4. use the observational measures of the selected objects to explore possible correlations
5. visualize astronomical images from the radio up to the high energy domain
6. display spectral energy distributions obtained from different photometric data sets

Schematic view of the scientific rationale of the tutorial: 
<img src="images/schema.png" alt="Drawing" style="width: 400px;"/>

## Step #1 - Obtaining all HESS sources from SIMBAD
How to obtain a catalogue of all published HESS sources from the astronomical database SIMBAD.

The High Energy Stereoscopic System (HESS) is a Cherenkov telescope array detecting cosmic gamma-rays in the GeV—TeV  range. Sources detected by HESS include supernova remnants, wind nebulae pulsar pulsars, active galactic nuclei and others. The number of detected sources steadily grows and makes it possible to study certain sub-samples of the HESS catalogue. In the following, the HESS sources should be investigated with respect to their gamma-ray properties obtained from the second Fermi-LAT catalogue.


In [ ]:
from pathlib import Path
import numpy as np

# PLOTLY very powerfull & interactive -- Needs a panda kind of table to work
# https://plot.ly/python/plotly-express/
import pandas as pd
import plotly.express as px


from astroquery.simbad import Simbad
from astroquery.xmatch import XMatch
from astroquery.vizier import Vizier

import ipyaladin.aladin_widget as ipyal

from astropy import units as u
from astropy.table import Table, join, unique, hstack, vstack
from astropy.coordinates import SkyCoord


In [ ]:
# Check which are the default fields
Simbad.get_votable_fields()


In [ ]:
# Check the available fields in a Simbad astroquery
Simbad.list_votable_fields()


In [ ]:
# Get more information about the field coordinatse
Simbad.get_field_description('otype')


To check all available object types in Simbad go to http://simbad.u-strasbg.fr/simbad/sim-display?data=otypes

In [ ]:
# Add the field otype and check that it is now included
Simbad.add_votable_fields('otype')
# Add new columns with the RA and Dec in degrees (default cols are h:m:s and d:m:s respectively)
Simbad.add_votable_fields('ra(d)')
Simbad.add_votable_fields('dec(d)')

Simbad.get_votable_fields()


In [ ]:
# Query Simbad by catalog containing the keyword 'hess' and display the result in pretty notebook format
hess_simbad = Simbad.query_catalog('hess')
hess_simbad.show_in_notebook(display_length=3)


There are 96 HESS sources in Simbad database. 

Note that we have limited the display to visualise only the first three entries. You can easily navigate through all the entries and you can sort the current view of the table by selecting a particular column.

You can also filter entries using the "Search:" botton, try e.g. to filter the table by HMXB. 
How many HESS sources in Simbad are classified as HMXB? And as SNR?

## Step #2 - Projecting HESS sources onto the Fermi sky
Display the HESS sources in Galactic coordinates and overlay it on a view of an all-sky image survey conducted with Fermi LAT.

In [ ]:
# Display the Fermi color HiPS
aladin = ipyal.Aladin(survey='P/Fermi/color')
aladin


In [ ]:
# Display the sources on top
aladin.add_table(hess_simbad)


The HESS sources are overlayed on top of the Fermi images. If you don't see them, zoom out and move around the sky to locate them around the Galactic Plane. 
By selecting a source you will see the main Simbad information of that source at the bottom of the AladinLite widget. You can change the image by selecting the stack layer. 

In [ ]:
# Select the "SNR" sources and overlay them
# Note that characters in python need that extra "b"
idx_SNR = hess_simbad['OTYPE'] == [b'SNR']
aladin.add_table(hess_simbad[idx_SNR])


## Step #3 - Cross-identify HESS and Fermi LAT sources
Here we will cross-correlate the HESS source list with the complete set of objects appearing in the *3rd Fermi LAT catalogue*. 

Then we use the CDS XMatch service (http://cdsxmatch.u-strasbg.fr/) via the astroquery.XMatch.query module to look for spatial crossmatches

The cross-correlation will create a new, sub-catalogue of all HESS sources found in Simbad that also appear in the 3rd Fermi-LAT catalogue. 

The Fermi Gamma Ray Space Telescope was launched in 2008 and observes astronomical objects across the 10 keV – 300 GeV band. The main instrument of the Fermi satellite is the Large Area Telescope (LAT), an imaging high-energy gamma-ray telescope covering the spectral range of 20 MeV – 300 GeV. The LAT has a very wide field of view, covering 20% of the sky at any time and scanning the complete sky every three hours. In the third Fermi-LAT catalogue (3FGL) all identified Fermi sources after 4 years of continuous survey were published together with basic measurements such as photometric fluxes in different spectral bands, an estimated gamma-ray spectral slope, or a variability flag.

In the following cross-identification process, it is assumed that the HESS source and its Fermi counterpart have an apparent angular distance of less then 1°, which is guided by the average angular resolution of the Fermi-LAT across the gamma-ray waveband. The angular resolution of the Fermi-LAT drops from 3° at 100 MeV to 0.04° at 100 GeV. Choosing a different limit for the angular distance thus favors associations with more objects at the low or at the high energy end of the Fermi-LAT band.

Find the VizieR name of the 3rd Fermi LAT catalogue. 

In [ ]:
catalog_list = Vizier.find_catalogs('Fermi LAT catalog')
for k, v in catalog_list.items():
    print(k, ': ', v.description)


Check the list of cadidate catalogs and select the one that interests us, i.e. "Fermi LAT third source catalog (3FGL) (Acero+, 2015)": **J/ApJS/218/23**. VizieR might have several tables associated to this catalog. 
The next step is to see if the catalog has more than one table associated. 
We display table names and metadata:

In [ ]:
Vizier.ROW_LIMIT = -1
catalog = Vizier.get_catalogs('J/ApJS/218/23')
print(catalog)
print('Metadata of each table: ')
print(catalog[0].meta)


There are three tables, each with a different number of columns and rows. To see those of interest, I display their description.

In [ ]:
for i in range(len(catalog)):
    print(catalog[i].meta['name'], ': ', catalog[i].meta['description'])
# for tab in catalog:
#    print(tab.meta['name'], ': ' ,tab.meta['description'])


I visualise the first element of each table to have an idea of their content. 

In [ ]:
table4 = catalog[0]
table4[0]


In [ ]:
table8 = catalog[1]
table8[0]


In [ ]:
table3 = catalog[2]
table3[0]


**J/ApJS/218/23/table4** and **J/ApJS/218/23/table8** contain positional and spectral information respectively. 
The first table has several entries for the same source, so before performing the match restric the table to **unique** entries for the same source. We then match the tables based on the names in column **_3FGL** to get all information in a single table. 

In [ ]:

table4 = unique(table4, keys='_3FGL')
FERMI = join(table4, table8, keys='_3FGL',
             join_type='inner', metadata_conflicts='silent')
print('Number of matches :', len(FERMI))
FERMI[0]


We have a new table with **3034 FERMI sources with spatial and spectral information**. We move on to cross-match this table with the HESS-Simbad sources. This time the match will be based on positions. 

Lets have a look again a the hess_simbad table and it's content.

In [ ]:
hess_simbad[0:5]


In [ ]:
# Create a skycoord object from the RA_d and DEC_d columns
c_hess_simbad = SkyCoord(hess_simbad['RA_d'], hess_simbad['DEC_d'])
c_fermi = SkyCoord(FERMI['RAJ2000'], FERMI['DEJ2000'])


In [ ]:
# Set the maximum separation
max_sep = 1.0 * u.deg
# For each row in the first catalog, look for the closest neighbor
idx, d2d, d3d = c_hess_simbad.match_to_catalog_sky(c_fermi, 1)
sep_constraint = d2d < max_sep

# limit the tables to matches within the maximum separation
hess_simbad_matches = hess_simbad[sep_constraint]
fermi_matches = FERMI[idx[sep_constraint]]

# Look at the number of matches in each table within the maximum separation
print('Number of possible matches:', len(hess_simbad_matches))
print('Number of possible matches:', len(fermi_matches))


There are **83 HESS-Simbad-FERMI** possible matches within 1 degree for each row in the first table! 
Tables **hess_simbad_matches** and **fermi_matches** are the matched sources in hess_simbad and fermi, respectively, which are separated by less than 1 deg. But ideally we would like to have the result as a single table including the angular separation between the matches. Let's go do that.

In [ ]:
# Add the separation to the fermi Table and concatenate the matched tables
fermi_matches['Ang_Sep'] = d2d[sep_constraint]
hess_simbad_fermi = hstack(
    [hess_simbad_matches, fermi_matches], metadata_conflicts='silent')

print('Number of possible matches', len(hess_simbad_fermi))


Note that the same Fermi source can be associated to different HESS sources. You can investigate those possible matches within 1deg (recommended) or limit to have unique entries. For the purpose of the exercice we choose option 2, but in general, we recommend to look at all possible matches to see which might be the true association (if any). 

In [ ]:
# Limit to unique 3FGL sources, this will keep the first row of each duplicate
# Note that one could sort them before that
hess_simbad_fermi.sort('Ang_Sep')
hess_simbad_fermi = unique(hess_simbad_fermi, keys='_3FGL', keep='first')
hess_simbad_fermi.show_in_notebook(display_length=3)


There are 65 FERMI-HESS-Simbad matches. Some of these associations are of known nature. 

Among the list of catalogs in VizieR that we listed before **J/ApJ/810/14** : *Third catalog of LAT-detected AGNs (3LAC) (Ackermann+, 2015)* lists 3LAC sources classified as AGNS. Let's have a look at the catalogue in question:


In [ ]:
Vizier.ROW_LIMIT = -1
catalog = Vizier.get_catalogs('J/ApJ/810/14')
for tab in catalog:
    print(tab.meta['name'], ': ', tab.meta['description'])


Table **J/ApJ/810/14/table10** lists the properties of very-high-energy ANGs. Let's have a look at that table:

In [ ]:
table10 = catalog[3]
table10.show_in_notebook(display_length=3)


There are no coordinates, only names are given in column **_3FGL**. Let's see how many of these AGNs are within our table of matches **hess_simbad_fermi**:

In [ ]:
hess_simbad_fermi_AGN = join(hess_simbad_fermi, table10,
                             keys='_3FGL', join_type='inner', metadata_conflicts='silent')
print('Number of matches :', len(hess_simbad_fermi_AGN))
hess_simbad_fermi_AGN[0]


There are **5 AGNs** ! Let's have a look at their properties and see how they compare with the rest of the population. 

## Step #4 - Searchig for relative locations in a "high-energy color-color diagram"
*How to trace various object properties of a given catalogue against each other and to point out specific sub-samples in the resulting plot.*

The next goal is to look for correlations between different high-energy gamma-ray colors of the HESS sources that are also detected by Fermi. In stellar astronomy, tracing optical colors against each other gives a rather robust classification of different star types without having to do (more ambitious) spectroscopy. In the HESS-FERMI catalogue there is not much spectroscopic information available, but the different fluxes in F1—F5 coming from the 3FGL part can be used to eventually discriminate different sub-samples of the catalogue. Since a color is basically an observed flux ratio between two bands, constructing a color-color diagram has the great advantage to be independent of the distance to the objects.

The “color-color” diagram to be computed involves the following high-energy gamma-ray fluxes as measured by Fermi-LAT:
- F1: flux over 100 – 300 MeV
- F2: flux over 300 - 1000 MeV
- F3: flux over 1 - 3 GeV
- F4: flux over 3 - 10 GeV
- F5: flux over 10 - 100 GeV


Merge all the tables in one, but keeping their origin by adding a column with name **Source**. Add a column for the object type **TYPE**. 

In [ ]:
# Create a column with name Source to know  which will contain types as "All" for FERMI,
FERMI['Source'] = 'FGL3'
hess_simbad_fermi['Source'] = 'HESS-Simbad-3FGL'
hess_simbad_fermi_AGN['Source'] = 'AGNs-3LAC'


In [ ]:
# Create a column with name TYPE which will contain types as "All" for FERMI,
# Have to change to str since OTYPE is of object type
FERMI['TYPE'] = 'ALL'
hess_simbad_fermi['TYPE'] = hess_simbad_fermi['OTYPE'].astype(str)
hess_simbad_fermi_AGN['TYPE'] = 'AGN'


In [ ]:
from astropy.table import vstack
# Merge the tables vertically using vstack
Data = vstack([FERMI, hess_simbad_fermi, hess_simbad_fermi_AGN],
              metadata_conflicts='silent')


In [ ]:
# Create two new columns with the flux ratios
Data['F2_F1'] = Data['F2']/Data['F1']
Data['F4_F3'] = Data['F4']/Data['F3']


Save the file into for instance csv format and into a Panda kind of table to be able to use Plotly

In [ ]:
# Write Table to csv format and to panda format
Data.write('Data' / Path('Data_FERMI-HESS-SIMBAD-AGN.csv'), overwrite=True)
Data = Data.to_pandas()


And now lets plot using Plotly:
- Flux ratios F2/F1 vs F4/F3
- In a log. scale 
- Color coded according to the Source 
- Using fancy markers 
- Adding a legend

In [ ]:
fig = px.scatter(Data, x='F2_F1', y='F4_F3', log_x=True,
                 log_y=True, color='Source', hover_name='TYPE')
fig.update_traces(marker=dict(size=12, line=dict(
    width=2, color='DarkSlateGrey')), selector=dict(mode='markers'))
fig.show()


Note that when you move around with the mouse you get the values for each of the sources. You can activate or deactivate a type of Source in the plot by selecting it in the legend. You can zoom in and out, you can export as a png... and plenty more. Play a little bit with it. 

Do the same as before but now color coded by object type. 

In [ ]:
# Plot same thing but color coded by the TYPE of object
fig = px.scatter(Data, 'F2_F1', y='F4_F3',
                 log_x=True, log_y=True, color='TYPE')
fig.update_traces(marker=dict(size=12, line=dict(
    width=2, color='DarkSlateGrey')), selector=dict(mode='markers'))
fig.show()


## Step #5 – Display the location of interesting sources and search multi-wavelength data around them

*How to obtain multi-wavelength images for the source you are interested in.* 

After isolating the 5 SIMBAD-confirmed BL Lac objects on the gamma-ray “color-color” diagram, this last step focuses on one particular object: **PKS 2155-304, a peculiar BL Lac object known to be very variable on a wide spectral range and to show recurrent outbursts**. A BL Lac object is a sub-type of active galactic nuclei, where the central engine around the accreting supermassive black hole launches magnetized, ballistic jets that reach out to kiloparsec scales while being nearly aligned with the line of sight of the observer. The emission from the jets is modeled as relativistically beamed synchrotron-self-Compton (SSC) emission produced in the hot plasma of the ejected jet material. The very hot electrons produce synchrotron emission that peaks in the radio band and then the very same electrons comptonize the synchrotron photons and up-scatter them to gamma-rays and beyond. However, in reality the question of the seed photons and the structure of the jet are still under investigation. A simple SSC interpretation of the broad band spectrum is challenged by the partially correlated variability seen between different spectral bands.

Broad-band spectrum reaching from the radio to the high-energy band
 (H.E.S.S., Fermi-LAT collaborations, ApJ 696, L150)
 
<img src="images/SED.png" alt="Drawing" style="width: 400px;"/>

In the following, this tutorial shows how to obtain multi-waveband imaging for this source (PKS 2155-304).

Now, let's plot again AladinLite with the catalog of AGNs on top on the Fermi image and centering around PKS 2155-30 and setting a field of view of 10 deg.

In [ ]:
obj = 'PKS 2155-304'
aladin = ipyal.Aladin(target=obj, survey='P/Fermi/color', fov=10)
aladin


In [ ]:
aladin.add_table(hess_simbad_fermi)
aladin.add_table(hess_simbad_fermi_AGN)


In [ ]:
sc = SkyCoord.from_name(obj)
sed_url = f'https://vizier.cds.unistra.fr/viz-bin/sed?-c={sc.icrs.ra.deg},{sc.icrs.dec.deg}&-c.rs=5'
sed = Table.read(sed_url, format='votable')

sed.show_in_notebook(display_length=3)


In [ ]:
sed['nufnu'] = sed['sed_freq']*sed['sed_flux']*1e-17
cc = 3e18
sed['lambda'] = cc/sed['sed_freq']/1e9
sed_p = sed.to_pandas()
fig = px.scatter(sed_p, x='lambda', y='nufnu',
                 log_x=True, log_y=True, color='_tabname')
fig.update_traces(marker=dict(size=10,
                              line=dict(width=1, color='DarkSlateGrey')),
                  selector=dict(mode='markers'))

fig.update_layout(xaxis_title='Wavelength [Å]', yaxis_title='νFν',
                  title={'text': 'Based on VizieR photometric Viewer',
                         'y': 0.95, 'x': 0.4,
                         'xanchor': 'center',
                         'yanchor': 'top'})

fig.show()
